# Sample Use Case for Creating and Searching on Tags

- In this example use case, 3 instances are created an named Groceries, Rent and Going out, with each representing a log of the different expenses a person might use to track their spending.

- A tag is created and named Expenses, and it is applied to two of the instances, with the last one being tagged using the [Update Tag](#update-tag) request.

- The `/search` endpoint is used to show how to search for instances based on the tag that was applied to them (in this case, Expenses)

- The other requests show how we can get the tag based on different criteria (`tag_name` and `tag_id`), update the tag (the example updates the tag's name), untag an instance by updating the tag's resources, and delete it - all using the `/tags` endpoint

- The final request uses the `/deletions` endpoint to delete the instances that we created earlier

- The two requests labeled Delete can be used to cleanup your server (assuming variable values have not changed)

- Running [Update Tag](#update-tag) will update the tag's name, as well as the `tag_name` variable.
Quick Links:

- [Imports and Global Variables](#imports)
- [Get Access Token](#authentication)
- [Create Instances](#create-instances)
- [Create Tag and Apply to Instances](#create-tag)
- [Search Based on `tag_name`](#search)
- [Get Created Tag by name](#get-tag-by-name)
- [Get Created Tag by ID](#get-tag-by-id)
- [Update Tag](#update-tag)
- [Remove an Instance from a Tag](#untag)
- [Delete Tag](#delete-tag)
- [Delete Created Instances](#delete-instances)

**NOTE**: Please make sure you run [Imports and Global Variables](#imports) before executing anything else in this notebook. Also ensure you are authenticated by running [Get Access Token](#authentication).


## **Imports and Global Variables** <a id='imports'></a>

**Run this cell before any of the others**

- Imports packages and sets variables that will be used throughout the notebook

- Update `sasserver` to your server url

**Note**: Running this cell will reset the `tag_id` and `instance_ids` variables.

In [ ]:
import json
import requests

sasserver = "https://your_server"

tag_id = ""
instance_ids = []

## Get Access Token <a id='authentication'></a>

- This sets the `access_token` that will be used in all of the calls in this use case.

- Edit the payload object with a username and password.

- Run this as needed since the token will expire after a certain amount of time.


In [ ]:
url = sasserver + "/SASLogon/oauth/token#password"

payload = {
    'grant_type': 'password',
    'username': 'your_username',
    'password': 'your_password'
}

headers = {
    'Authorization': 'Basic c2FzLmVjOg==',
}

response = requests.request(
    "POST", url, headers=headers, data=payload, verify=False)

print(f"Status code: {response.status_code}")

if response.status_code == 200:
    print("Successfully authenticated")

    response = response.json()
    access_token = response["access_token"]
    access_token = f"Bearer {access_token}"
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

## Create Instances <a id='create-instances'></a>

- Creates the instances that will be tagged.

- For this use case, three instances are created - named Rent, Groceries and Going out.  These instances will be used to demonstrate tagging. Modify the instances as needed to fit your use case.

- Once the instances are created, the server will assign an id to each. The ids are stored in `instance_ids`, which is used in [Create Tag and Apply to Instances](#create-tag), and [Delete Created Instances](#delete-instances).


In [ ]:
url = sasserver + "/catalog/instances"

entities = [
    {
        "instanceType": "entity",
        "name": "Rent",
        "label": "A log of rent expenses",
        "definition": "casTable",
        "resourceId": "/some~path/some~other~path/rent/apartment"
    },
    {
        "instanceType": "entity",
        "name": "Groceries",
        "label": "A log of grocery expenses",
        "definition": "casTable",
        "resourceId": "/some~path/some~other~path/groceries"
    },
    {
        "instanceType": "entity",
        "name": "Going out",
        "label": "A log of expenses when going out",
        "definition": "casTable",
        "resourceId": "/some~path/some~other~path/going~out/restaurants"
    }
]

payload = json.dumps({"entities": entities})

headers = {
    'Content-Type': 'application/vnd.sas.metadata.instance.archive+json',
    'Authorization': access_token
}

response = requests.request(
    "POST", url, headers=headers, data=payload, verify=False)

print(f"Status code: {response.status_code}")

if response.status_code == 200:
    print("Instances created successfully")

    response = response.json()
    entities = response["entities"]
    instance_ids = [entity["id"] for entity in entities]

    print("Instance IDs:", instance_ids)
    print(f"Response:\n{json.dumps(response, indent=2)}")

else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

## Create Tag and Apply to Instances <a id='create-tag'></a>

- Creates the tag and applies it to two of the three instances. The third one is added when running [Update Tag](#update-tag). An instance can be removed from a tag using [Remove Instance from a Tag](#untag).

- The `tag_name` variable can be set to any value.

In [ ]:
url = sasserver + "/catalog/tags"

tag_name = "Expenses"

tag = {
    "name": tag_name,
    "members": {
        "type": "id",
        "template": "/catalog/instances/{id}",
        "resources": instance_ids[:-1]
    }
}

payload = json.dumps(tag)

headers = {
    'Content-Type': 'application/json',
    'Authorization': access_token,
}

response = requests.request(
    "POST", url, headers=headers, data=payload, verify=False)

print(f"Status code: {response.status_code}")

if response.status_code == 201:
    print(
        f"Tag \"{tag_name}\" created, and applied to instances \"{entities[0]['name']}\" and \"{entities[1]['name']}\"")

    tag_etag = response.headers["ETag"]
    last_modified = response.headers["Last-Modified"]

    response = response.json()

    tag_name = response["name"]

    tag_id = response["id"]

    print("Tag ID:", tag_id)
    print(f"Response:\n{json.dumps(response, indent=2)}")
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

## Search Based on tag_name <a id='search'></a>

- Gets the instances that have been tagged, based on a tag's name

- Run after [Post Tag](#create-tag)/[Update Tag](#update-tag)/[Remove Instance from a Tag](#untag) to check the currently tagged instances.

- This uses the `/search` endpoint

In [ ]:
url = sasserver + f"/catalog/search?q=Tags: \"{tag_name}\""

headers = {
    'Authorization': access_token,
}

response = requests.request(
    "GET", url, headers=headers, data=payload, verify=False)

print(f"Status code: {response.status_code}")

if response.status_code == 200:
    print("Search based on tag_name succeeded")

    response = response.json()

    tagged_entities = [entity["name"] for entity in response["items"]]

    print(f"Tag \"{tag_name}\" is currently applied to:")
    print("  " + "\n  ".join(tagged_entities))
    print(f"Response:\n{json.dumps(response, indent=2)}")
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

## Get Created Tag by name <a id='get-tag-by-name'></a>

- Gets the tag based on the tag's name, `tag_name`. This uses the `/tags` endpoint and a filter based on the name of the tag.

In [ ]:
url = sasserver + "/catalog/tags" + f"?filter=eq(name,\"{tag_name}\")"

headers = {
    'Accept': 'application/vnd.sas.collection+json',
    'Accept-Item': 'application/vnd.sas.metadata.tag.summary+json',
    'Authorization': access_token,
}

print(f"Current value of tag_name: \"{tag_name}\"")

response = requests.request(
    "GET", url, headers=headers, data=payload, verify=False)

print(f"Status code: {response.status_code}")

if response.status_code == 200:
    print("Get tag by filtering on tag_name succeeded")

    response = response.json()
    print(f"Response:\n{json.dumps(response, indent=2)}")

    tag = response["items"][0]
    tag_id = tag["id"]
    tag_name = tag["name"]
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

## Get Created Tag by ID

- This is similar to [Get Created Tag by name](#get-tag-by-name), except it uses the `tag_id` to retrieve the tag.


In [ ]:
url = sasserver + "/catalog/tags/" + tag_id

headers = {
    'Accept': 'application/vnd.sas.metadata.tag+json',
    'Authorization': access_token,
}

response = requests.request(
    "GET", url, headers=headers, data=payload, verify=False)

print(f"Status code: {response.status_code}")

if response.status_code == 200:
    print("Get created tag by id succeeded")

    tag_etag = response.headers["ETag"]

    response = response.json()
    print(f"Response:\n{json.dumps(response, indent=2)}")

    tag_id = response["id"]
    tag_name = tag["name"]
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

## Update Tag <a id='update-tag'></a>

- This request demonstrates how to update a tag. The tag's name is updated by appending "\_updated" to the original name.


In [ ]:
url = sasserver + "/catalog/tags/" + tag_id

updated_tag = {
    "id": tag_id,
    "name": f"{tag_name}_updated",
    "members": {
        "type": "id",
        "template": "/catalog/instances/{id}",
        "resources": instance_ids
    }
}

payload = json.dumps(updated_tag)
headers = {
    'Content-Type': 'application/vnd.sas.metadata.tag+json',
    'Accept': 'application/vnd.sas.metadata.tag+json',
    'If-Match': tag_etag,
    'Authorization': access_token,
}

response = requests.request(
    "PUT", url, headers=headers, data=payload, verify=False)

print(f"Status code: {response.status_code}")

if response.status_code == 200:
    print("Tag updated successfully")

    tag_etag = response.headers["ETag"]
    last_modified = response.headers["Last-Modified"]

    response = response.json()
    tag_id = response["id"]
    tag_name = response["name"]

    print(
        f"The name of the tag was updated to \"{tag_name}\" and the instance \"{entities[-1]['name']}\" was tagged")
    print(f"Response:\n{json.dumps(response, indent=2)}")
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

## Remove an Instance from a Tag <a id='untag'></a>

- This request demonstrates how to remove an instance from a tag. This same request can be used to add an instance to a tag by updating the tag body "op" to "add".

- Removes the instance from the members.resources property of the tag. 
- Run [Search Based on tag_name](#search) before and after this call to see that the instance was removed.


In [ ]:
url = sasserver + "/catalog/tags" + "?onConflict=update"

patch_request = {
    "tags": {
        tag_id: [
            {
                "op": "remove",
                "path": "/members/resources",
                "value": entities[-1]["id"]
            }
        ]
    }
}
payload = json.dumps(patch_request)

headers = {
    'Content-Type': 'application/vnd.sas.metadata.patch.archive+json',
    'If-Unmodified-Since': last_modified,
    'Authorization': access_token,
}

response = requests.request(
    "PATCH", url, headers=headers, data=payload, verify=False)

print(f"Status code: {response.status_code}")

if response.status_code == 200:
    print(
        f"The instance \"{entities[-1]['name']}\" was untagged successfully")

    last_modified = response.headers["Last-Modified"]

    response = response.json()
    print(f"Response:\n{json.dumps(response, indent=2)}")

    # Update tag_etag after untagging instance
    url = sasserver + "/catalog/tags/" + tag_id
    headers = {
        'Accept': 'application/vnd.sas.metadata.tag+json',
        'Authorization': access_token,
    }
    response = requests.request(
        "GET", url, headers=headers, data=payload, verify=False)
    if response.status_code == 200:
        tag_etag = response.headers["ETag"]
        print("ETag updated successfully")
    else:
        print(
            f"Error updating ETag. Here is the response:\n{json.dumps(response.json(), indent=2)}")
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

## Delete Tag <a id='delete-tag'></a>

- Deletes a tag based on it's id.

- This and [Delete Created Instances](#delete-instances) can be run to clean up your server.

In [ ]:
url = sasserver + "/catalog/tags/" + tag_id

headers = {
    'Authorization': access_token,
}

response = requests.request(
    "DELETE", url, headers=headers, verify=False)

print(f"Status code: {response.status_code}")

if response.status_code == 204:
    print(f"Tag \"{tag_name}\" deleted successfully")
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

## Delete Created Instances <a id='delete-instances'></a>

- Deletes the instances that were created for this sample use case.
- This and [Delete Tag](#delete-tag) can be run to clean up your server.


In [ ]:
url = sasserver + "/catalog/deletions"

resources = [
    f"/catalog/instances/{instance_id}" for instance_id in instance_ids]

payload = json.dumps({
    "version": 1,
    "resources": resources
})

headers = {
    'Content-Type': 'application/json',
    'Authorization': access_token,
}

response = requests.request(
    "POST", url, headers=headers, data=payload, verify=False)

print(f"Status code: {response.status_code}")

if response.status_code == 200:
    print("Entities deleted successfully")

    response = response.json()
    print(f"Response:\n{json.dumps(response, indent=2)}")
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")